<a href="https://colab.research.google.com/github/ConorDataHub/AI/blob/main/TestAStarAlgorithm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [5]:
initial_state = [0, 3, 5, 7,
                 1, 11, 13, 15,
                 9, 4, 6, 8,
                 2, 10, 12, 14]

In [6]:
goal_state = [1, 3, 5, 5,
              9, 11, 13, 15,
              2, 4, 6, 8,
              10, 12, 14, 0]

In [7]:
size = 16                 # this is the size of the list
dimension = 4             # dimension is the total number of of elements 15+1 (square root of 16)

In [8]:
def check_diff_score(state):        # This function takes a state as an array and its purpose is to keep the score of the number of tiles that are out of place at each stage of the algorithm
  diff = 0
  for i in range(size):
      current_item = state[i]
      if current_item != 0 and current_item != goal_state[i]:
          diff +=1
  return diff


In [9]:
print(initial_state)
print(goal_state)
diff = check_diff_score(initial_state)
print(diff)

[0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14]
[1, 3, 5, 5, 9, 11, 13, 15, 2, 4, 6, 8, 10, 12, 14, 0]
7


In [34]:
def print_state(state):               # Function to print state as matrix
  for i in range(size):               # Range function used to work with the array index, size is 16 which is the number of items in the states 0-15
    if i % dimension == 0:
      print("")                      # This print statement is printed before every line
    print(state[i], "", end="")      # Prints the current value of the state as a string
  print("\n")


In [35]:
print_state(initial_state) # example of the format that is printed


0 3 5 7 
1 11 13 15 
9 4 6 8 
2 10 12 14 



In [42]:
# These dictionary will have the list of items inside of it - every value has a key associated with it. The hashing helps to reduce time complexity compared to lists O(1) v O(n)

matrix_to_1D_mapping = {}               # mapping the index of a 2 dimensional array to a 1 dimensional index 
index_to_2D_mapping = {}                # mapping the index of a 1 dimensional array to a 2 dimensional index                           

In [43]:
index = 0
for row in range(dimension):
  for column in range(dimension):
    matrix_to_1D_mapping[(row, column)] = index
    index_to_2D_mapping[index] = (row, column)
    index += 1


In [44]:
print(matrix_to_1D_mapping)
print(index_to_2D_mapping)

{(0, 0): 0, (0, 1): 1, (0, 2): 2, (0, 3): 3, (1, 0): 4, (1, 1): 5, (1, 2): 6, (1, 3): 7, (2, 0): 8, (2, 1): 9, (2, 2): 10, (2, 3): 11, (3, 0): 12, (3, 1): 13, (3, 2): 14, (3, 3): 15}
{0: (0, 0), 1: (0, 1), 2: (0, 2), 3: (0, 3), 4: (1, 0), 5: (1, 1), 6: (1, 2), 7: (1, 3), 8: (2, 0), 9: (2, 1), 10: (2, 2), 11: (2, 3), 12: (3, 0), 13: (3, 1), 14: (3, 2), 15: (3, 3)}


In [61]:
# Need to store open and closed lists

#Function to change list to string
def get_state_id(state):
  string_list = []
  for item in state:
    string_list.append(str(item))
  return ", ".join(str(v) for v in state)               # For every item inside the list - cast that item to a string and join with a comma - this is so we can use a string to query the distionary

In [65]:
print(initial_state)
id = get_state_id(initial_state)
print(id)

[0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14]
['0', '3', '5', '7', '1', '11', '13', '15', '9', '4', '6', '8', '2', '10', '12', '14']
0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14


In [68]:
def get_node_state(id):
  state = []
  split_list = id.split(",")
  for item in split_list:
    state.append(int(item))
  print(split_list)
  return state

In [69]:
get_node_state("0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14")

['0', ' 3', ' 5', ' 7', ' 1', ' 11', ' 13', ' 15', ' 9', ' 4', ' 6', ' 8', ' 2', ' 10', ' 12', ' 14']


[0, 3, 5, 7, 1, 11, 13, 15, 9, 4, 6, 8, 2, 10, 12, 14]

In [70]:
# Need two lists/dictionaries, one for open nodes and one for closed nodes(nodes of previous states)

opened_nodes = {}
closed_nodes = {}

# Current state is assigned the value of the initial state. The .copy method is used so that the original list will remain unchanged
current_state = initial_state.copy()
# Depth tracks the depths of the nodes, starting from 0
depth = 0

# This method counts the number of tiles currently out of place
initial_score = check_diff_score(current_state)
print("initial score: ", initial_score)

if(initial_score == 0):
  print("Puzzle solved")
else:
  pass


initial score:  7
